In [1]:
## Connect to gg driver
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/AI Tutor/CV/Topic2: Object Classification/day06

/content/drive/.shortcut-targets-by-id/1g_hBCGxmI5lTFXyvD-igJcroxCjDzObt/AI Tutor/CV/Topic2: Object Classification/day06


In [11]:
%%capture
!pip install onnxruntime
!pip install onnxruntime-gpu
!pip install netron
!pip install onnx
!pip install pyngrok

### Common function usage


In [1]:
import torch
import torch.onnx
import onnxruntime
import numpy as np

In [2]:
def convert_pytorch2onnx(model, input_samples, path_onnx, mode = 'float32bit', device = 'cuda'):
    if mode == 'float16bit':
        print("float16bit")
        model.float()
        model.half()  # Chuyển mô hình sang float16
        input_samples = input_samples.half()
    model.to(device)
    model.eval()
    input_samples = input_samples.to(device)
    torch.onnx.export(
        model, # Model load bằng pytorch
        input_samples, # input với kích thước mong muốn
        path_onnx, # path_onnx bạn lưu
        verbose=False,  # Hiển thị thông báo trong quá trình chuyển đổi
        opset_version=12, # Đọc xem mỗi version sẽ có hỗ trợ onnx cho lớp nào
        do_constant_folding=True , # Dùng Constant-folding optimizer giúp cải thiện tốc độ và dung lượng
        input_names = ['images'],   # the model's input names
        output_names = ['output'], # the model's output names
        dynamic_axes={
                'images' : {0 : 'batch_size',},
                'output' : {0 : 'batch_size'}
            }
        )

def load_onnx_model(path_onnx, providers=['CUDAExecutionProvider', 'CPUExecutionProvider']):
    # Create an ONNX Runtime inference session for the ONNX model
    ort_session = onnxruntime.InferenceSession(
        path_onnx,
        providers=providers
        )
    return ort_session

def onnx_infer(ort_session, input_data):
    ort_inputs = {ort_session.get_inputs()[0].name: input_data}
    ort_output = ort_session.run(None, ort_inputs)
    return ort_output

## Convert model to onnx

In [3]:
## Load model pytorch
model = torch.load("models/torch/resnet18.pth")

In [5]:
mode ='float32bit'
path_onnx = f'models/onnx/vgg19_{mode}.onnx'
input_samples = torch.randn(16, 3, 224, 224)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
convert_pytorch2onnx(
    model=model,
    input_samples=input_samples,
    path_onnx=path_onnx,
    mode=mode,
    device=device
)


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [6]:
mode ='float16bit'
path_onnx = f'models/onnx/vgg19_{mode}.onnx'
input_samples = torch.randn(16, 3, 224, 224)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
convert_pytorch2onnx(
    model=model,
    input_samples=input_samples,
    path_onnx=path_onnx,
    mode=mode,
    device=device
)


float16bit
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [7]:
batch_size = 4
input_data = torch.randn(batch_size, 3, 224, 224)


In [8]:
onnx_float16 = load_onnx_model('models/onnx/vgg19_float16bit.onnx')
input_16bit = input_data.half()
input_numpy = input_16bit.numpy()
onnx_infer(onnx_float16, input_numpy)

c:\Users\DAT NGUYEN\AppData\Local\Programs\Python\Python310\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


[array([[-4.63  ,  0.0668,  0.6685, -5.613 ,  6.2   , -1.191 ],
        [-4.78  , -0.1383,  0.8306, -5.64  ,  6.37  , -1.162 ],
        [-4.293 , -0.135 ,  0.644 , -5.574 ,  5.973 , -1.134 ],
        [-4.375 ,  0.4526,  0.4885, -5.13  ,  5.86  , -1.566 ]],
       dtype=float16)]

In [9]:
onnx_float32 = load_onnx_model('models/onnx/vgg19_float32bit.onnx')
input_numpy = input_data.numpy()
onnx_infer(onnx_float32, input_numpy)

[array([[-4.6215286 ,  0.07158978,  0.6677648 , -5.6022606 ,  6.190439  ,
         -1.1997516 ],
        [-4.772766  , -0.1336788 ,  0.8294039 , -5.630218  ,  6.364069  ,
         -1.1685611 ],
        [-4.286834  , -0.13217452,  0.6436262 , -5.5651846 ,  5.966695  ,
         -1.140068  ],
        [-4.367727  ,  0.45466512,  0.48828512, -5.1184015 ,  5.849763  ,
         -1.5732332 ]], dtype=float32)]

In [10]:
## Load model pytorch
model = torch.load("models/torch/resnet18.pth")
model.to('cuda')
with torch.no_grad():
    out = model(input_data.to('cuda'))
out

tensor([[-4.6211,  0.0711,  0.6674, -5.6047,  6.1900, -1.1971],
        [-4.7738, -0.1344,  0.8293, -5.6337,  6.3649, -1.1655],
        [-4.2869, -0.1328,  0.6434, -5.5676,  5.9665, -1.1374],
        [-4.3673,  0.4544,  0.4883, -5.1202,  5.8495, -1.5718]],
       device='cuda:0')

In [11]:
#!pip install netron
import netron
netron.start('models/onnx/vgg19_float32bit.onnx', 8081)

Serving 'models/onnx/vgg19_float32bit.onnx' at http://localhost:8081


('localhost', 8081)

In [35]:
# https://netron.app/
# chạy online lấy weight onnx upload lên